### Imports

In [2]:
import zipfile
import fiftyone as fo
import fiftyone.zoo as foz
import shutil
import os

Migrating database to v0.23.2


### Download Data

In [7]:
# Fet data
data = foz.load_zoo_dataset(
    "coco-2017",
    dataset_name="coco-sample",
    split="validation",
    label_field=["detections"],
    classes=["car", "person"],
    max_samples=100,
)

Found annotations at '/home/codespace/fiftyone/coco-2017/raw/instances_val2017.json'
 100% |██████████████████| 100/100 [26.5s elapsed, 0s remaining, 3.7 images/s]      
Writing annotations for 100 downloaded samples to '/home/codespace/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/home/codespace/fiftyone/coco-2017/info.json'
Loading existing dataset 'coco-sample'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [8]:
# Move the dataset to the new location
shutil.move('/home/codespace/fiftyone/coco-2017/validation/data', '/workspaces/obj_det/data')
shutil.move('/home/codespace/fiftyone/coco-2017/validation/labels.json', '/workspaces/obj_det/data')
shutil.move('/home/codespace/fiftyone/coco-2017/info.json', '/workspaces/obj_det/data')

# Rename folder
os.rename('/workspaces/obj_det/data/data', '/workspaces/obj_det/data/images')

Error: Destination path '/workspaces/obj_det/data/labels.json' already exists

### Download Model

In [9]:
mkdir /workspaces/obj_det/backend/model_store

mkdir: cannot create directory ‘/workspaces/obj_det/backend/model_store’: File exists


In [47]:
%%sh
wget https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth -O /workspaces/obj_det/backend/model_store/fasterrcnn_resnet50.pth

--2024-01-04 12:18:13--  https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
Resolving download.pytorch.org (download.pytorch.org)... 143.204.215.17, 143.204.215.87, 143.204.215.66, ...
Connecting to download.pytorch.org (download.pytorch.org)|143.204.215.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167502836 (160M) [binary/octet-stream]
Saving to: ‘/workspaces/obj_det/backend/model_store/fasterrcnn_resnet50.pth’

     0K .......... .......... .......... .......... ..........  0% 4.48M 36s
    50K .......... .......... .......... .......... ..........  0% 5.10M 33s
   100K .......... .......... .......... .......... ..........  0% 15.4M 26s
   150K .......... .......... .......... .......... ..........  0% 9.65M 23s
   200K .......... .......... .......... .......... ..........  0% 18.2M 21s
   250K .......... .......... .......... .......... ..........  0% 19.2M 18s
   300K .......... .......... .......... .......... ..........

### Archive Model

In [4]:
%%sh
torch-model-archiver \
--model-name fasterrcnn_resnet50 --version 1.0  \
--model-file ./backend/model.py \
--serialized-file /workspaces/obj_det/backend/model_store/fasterrcnn_resnet50.pth \
--export-path /workspaces/obj_det/backend/model_store \
--extra-files ./backend/index_to_name.json  \
--handler object_detector

In [1]:
%%sh
curl http://127.0.0.1:8080/predictions/fasterrcnn_resnet50 -T test.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  176k  100   322  100  176k     65  36890  0:00:04  0:00:04 --:--:--    87


[
  {
    "person": [
      724.263427734375,
      99.63356018066406,
      1765.712158203125,
      1865.07421875
    ],
    "score": 0.998004138469696
  },
  {
    "laptop": [
      430.6576843261719,
      1213.976806640625,
      1574.62548828125,
      1572.9703369140625
    ],
    "score": 0.9630018472671509
  }
]